In [83]:
filePaths = [
    ['/content/stats/0/test_fb11d - Copy.a3m', 45.8],
    ['/content/stats/1/test_fb11d.a3m', 46.2],
    ['/content/stats/2/test_fb11d.a3m', 47.5]
]

In [84]:
hydrophobic = dict({
    "K": 5.72,
    "N": 6.17,
    "D": 6.18,
    "E": 6.38,
    "P": 6.64,
    "Q": 6.67,
    "R": 6.81,
    "S": 6.93,
    "T": 7.08,
    "G": 7.31,
    "A": 7.62,
    "H": 7.85,
    "W": 8.41,
    "Y": 8.53,
    "F": 8.99,
    "L": 9.37,
    "M": 9.83,
    "I": 9.99,
    "V": 10.38,
    "C": 10.93,
    "-": 0,
    "X": 0
})

def transformHydro(seq):
  sum = 0
  for el in seq:
    value = hydrophobic.get(el)
    if value is not None:
      value = float(value)
    else:
      value = 0
    sum += value

  return sum

In [85]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-1):
    # Create a matrix to store the alignment scores
    n = len(seq1)
    m = len(seq2)
    score_matrix = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize the first row and column of the matrix with gap penalties
    for i in range(n + 1):
        score_matrix[i][0] = i * gap
    for j in range(m + 1):
        score_matrix[0][j] = j * gap

    # Fill in the matrix using dynamic programming
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                match_score = score_matrix[i - 1][j - 1] + match
            else:
                match_score = score_matrix[i - 1][j - 1] + mismatch

            delete_score = score_matrix[i - 1][j] + gap
            insert_score = score_matrix[i][j - 1] + gap

            score_matrix[i][j] = max(match_score, delete_score, insert_score)

    # Traceback to find the aligned sequences
    i, j = n, m
    alignment_seq1, alignment_seq2 = [], []

    while i > 0 or j > 0:
        if i > 0 and score_matrix[i][j] == score_matrix[i - 1][j] + gap:
            alignment_seq1.insert(0, seq1[i - 1])
            alignment_seq2.insert(0, '-')
            i -= 1
        elif j > 0 and score_matrix[i][j] == score_matrix[i][j - 1] + gap:
            alignment_seq1.insert(0, '-')
            alignment_seq2.insert(0, seq2[j - 1])
            j -= 1
        else:
            alignment_seq1.insert(0, seq1[i - 1])
            alignment_seq2.insert(0, seq2[j - 1])
            i -= 1
            j -= 1

    return score_matrix[n][m]

In [86]:
proteins = {}

for results in filePaths:
  path = results[0]
  score = results[1]

  file = open(path)
  print('Reading file "{}"'.format(path))
  lines = file.readlines()

  for line in lines:
    if line[:1] == '#':
      # Skip
      print('Header: {}'.format(line))
    elif line[:1] == '>':
      identifier = line.strip()
    else:
      # Add protein
      if identifier in proteins:
        proteins[identifier.strip()]['count'] += 1
        proteins[identifier.strip()]['score'] += score
      else:
        proteins[identifier.strip()] = {}
        proteins[identifier.strip()]['count'] = 1
        proteins[identifier.strip()]['sequence'] = line.strip()
        proteins[identifier.strip()]['score'] = score

  print("Total number of unique proteins: {}".format(len(proteins)))

Reading file "/content/stats/0/test_fb11d - Copy.a3m"
Header: #59,41,34	1,1,1

Total number of unique proteins: 49
Reading file "/content/stats/1/test_fb11d.a3m"
Header: #59,41,34	1,1,1

Total number of unique proteins: 69
Reading file "/content/stats/2/test_fb11d.a3m"
Header: #59,41,34	1,1,1

Total number of unique proteins: 82


In [87]:
# Calculate each average value
for protein in proteins:
  proteins[protein.strip()]['score'] = proteins[protein.strip()]['score'] / proteins[protein.strip()]['count']

In [88]:
results = open('/content/results.csv', 'w')

results.write('Identifier;Sequence;Score;Count;Hydro;Sequence_length;Distance\n')

for protein in proteins:
  line = ''

  line += protein
  line += ';'
  line += proteins[protein]['sequence']
  line += ';'
  line += str(proteins[protein]['score'])
  line += ';'
  line += str(proteins[protein]['count'])
  line += ';'
  line += str(transformHydro(proteins[protein]['sequence']))
  line += ';'
  line += str(len(proteins[protein]['sequence']))
  line += ';'
  line += str(needleman_wunsch('PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK', proteins[protein]['sequence']))
  line += '\n'

  print(line)
  results.write(line)

>101	102	103;PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASKAIAQIHICGRSDEQLCGRSDEQKETLIREGHFGIGGELASKAIAQIHILCGRSDEQKETLIREGHFGIGGELASK;46.5;3;1037.409999999999;134;-16

>101;PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK---------------------------------------------------------------------------;46.5;6;457.94000000000017;134;-16

>A0A255HIH8	56	0.327	3.998E-07	0	57	59	1	58	134;PTIQAYITAGQPPASRQRLIALLTDAVVASVDAPPEAVCVMLTEVPHEHYAVAGQPAS----------------------------------------------------------------------------;46.5;3;459.46;134;-94

>MGYP001021188106	60	0.258	1.649E-08	1	58	59	86	143	153;-VVTIELFQGRSLEAKRKMYAAIVRNLKAALDLDPADVRIVLVEATRENWAMGGKPASE---------------------------------------------------------------------------;46.65;2;452.2500000000001;134;-95

>MGYP001168304003	85	0.491	3.253E-17	0	58	59	11	69	74;PLIRVDMLEGRTPEQKRELIRELTETTSRVLGTPRERVRVVIYEVPKTHWGIGGTPVSE---------------------------------------------------------------------------;46.5;3;457.07000000000